In [65]:
import django, sys, os
sys.path.append('/home/max/software/django-tmv/tmv_mcc-apsis/BasicBrowser')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()
from pandas_ods_reader import read_ods
import pandas as pd
from scoping.models import *
import random
django.db.connection.close()

In [66]:
p = Project.objects.get(pk=178)
p

<Project: Regional Impacts Map>

In [67]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./jupyter-google-e4b72bf44320.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '10UE_YB-3eJd5H2BxPIzwve9VQOBXwDW_X1nGsKel12o'
wb = gc.open_by_key(spreadsheet_key)
ws = wb.worksheet("Final categories")

In [68]:
table = ws.get_all_values()
df = pd.DataFrame(table[1:], columns=table[0])
df.head()

,Individual impact categories,description,Heading,order,Include,System type,Category type,Source,AR5 Symbol Categories,AR5 Table Categories,AR6 System Categories,AR6 Impact Categories,Final categories
0,CO2 concentration,,0,21,1,01. Physical systems,Climate impacts,,,Physical systems,,,
1,Air or land surface temperature changes,,0,1,1,01. Physical systems,Climate impacts,Additional,,Physical systems,,,
2,Extreme temperature,,0,2,1,01. Physical systems,Climate impacts,Additional,,Physical systems,,,
3,Radiation,,0,13,1,01. Physical systems,Climate impacts,Additional,,Physical systems,,,
4,Changes in precipitation,,0,3,1,01. Physical systems,Climate impacts,Additional,,Physical systems,,,


In [69]:
imcat = Category.objects.get(pk=616)
dcat = Category.objects.get(pk=712)

In [70]:
Category.objects.filter(name__isnull=True,project=p).delete()

(0, {})

In [71]:
create_cats = True
ar5 = Category.objects.get(pk=468)
cat_number = 0
for i, row in df.iterrows():
    sys = row['System type'][4:]
    syscat = Category.objects.get(project=p,level=3,name__icontains=sys)
    if row['Include']=='1':
        cat_number+=1
    elif row['Heading']=='1':
        name = row['Individual impact categories']
        cname = f"{int(cat_number+1):02}.0 {name}"
        rpat = "[0-9]{2}.0 " + name.replace('(','\\(').replace(')','\\)') + "($|<)"
        for l in [4,6]:
            try:
                cat = Category.objects.get(project=p, level=l,name__iregex=rpat,title_only=True)
                cat.name=cname
                cat.equivalents.add(syscat)
                cat.parent_category = imcat
                cat.save()
                print(cat.name)
            except:
                print(f"couldn't find title cat {name}")
                if create_cats:
                    cat, created = Category.objects.get_or_create(project=p, level=l,name=cname,title_only=True)

        continue

    ar5name = re.sub("([0-9]*\W[0-9]* )","",row['AR5 Table Categories'],1)
    try:
        ar5t = Category.objects.get(project=p, level=12, name__icontains=ar5name)
    except:
        if create_cats:
            ar5t, created = Category.objects.get_or_create(project=p,level=12,name=ar5name, parent_category=ar5)
        else:
            print(ar5name)
            continue
            
            
    ar5t.name=row['AR5 Table Categories']
    
    name = row['Individual impact categories']
    cname = f"{int(cat_number):02} {name}"
    rpat = "[0-9]{2} " + name.replace('(','\\(').replace(')','\\)') + "($|<)"
    try:
        cat = Category.objects.get(project=p, level=4,name__iregex=rpat)
        cat.name=cname
    except:
        print(cname)        
        if create_cats:
            cat, created = Category.objects.get_or_create(project=p,level=4,name=cname)
            
    ### Hide or show category
    if row['Include'] == '0':
        if "<hidden>" not in cat.name:
            cat.name = cat.name+"<hidden>"
    else:
        if "<hidden>" in cat.name:
            cat.name = cat.name.replace('<hidden>','')
            
    
    cat.parent_category = imcat
    cat.description=name
    if row['description'] != "":
        cat.description = row['description']
        
    cat.equivalents.clear()
    cat.equivalents.add(syscat)
    cat.equivalents.add(ar5t)
    cat.save()
    
    try:
        cat = Category.objects.get(project=p, level=6,name__iregex=rpat)
        cat.name=cname
    except:
        print(cname)
        if create_cats:
            cat, created = Category.objects.get_or_create(project=p,level=6,name=cname)
            
    ### Hide or show category
    if row['Include'] == '0':
        if "<hidden>" not in cat.name:
            cat.name = cat.name+"<hidden>"
    else:
        if "<hidden>" in cat.name:
            cat.name = cat.name.replace('<hidden>','')
            
    cat.parent_category = dcat
    cat.description=name
    if row['description'] != "":
        cat.description = row['description']
    cat.equivalents.clear()
    cat.equivalents.remove(syscat)
    syscat = Category.objects.get(project=p,level=5,name__icontains=sys)
    cat.equivalents.add(syscat)
    cat.equivalents.add(ar5t)
    cat.save()

14.0 Oceans
14.0 Oceans
21.0 Rivers, lakes and soil moisture
21.0 Rivers, lakes and soil moisture
30.0 Mountains, snow and ice
30.0 Mountains, snow and ice
36.0 Marine & coastal
36.0 Marine & coastal
49 Biogeochemical flows (marine & coastal)
49 Biogeochemical flows (marine & coastal)
50 Other (marine & coastal)
50 Other (marine & coastal)
51.0 Terrestrial and freshwater
51.0 Terrestrial and freshwater
57 Biogeochemical flows (Terrestrial and freshwater)
57 Biogeochemical flows (Terrestrial and freshwater)
81 Other (human and managed systems)
81 Other (human and managed systems)


In [53]:
row

Individual impact categories    Terrestrial and freshwater
description                                               
Heading                                                  1
order                                                     
Include                                                  0
System type                                               
Category type                                             
Source                                                    
AR5 Symbol Categories                                     
AR5 Table Categories                                      
AR6 System Categories                                     
AR6 Impact Categories                                     
Final categories                                          
Name: 49, dtype: object

In [42]:
name = row['Individual impact categories']
cname = f"{int(cat_number+1):02} {name}"
rpat = "[0-9]{2}.0 " + name.replace('(','\\(').replace(')','\\)') + "($|<)"
rpat


'[0-9]{2}.0 Terrestrial and freshwater($|<)'

In [43]:
cat = Category.objects.get(project=p, level=4,name__iregex=rpat,title_only=True)

In [41]:
rpat = "[0-9]{2} " + name.replace('(','\\(').replace(')','\\)') + "($|<)"
print(rpat)
try:
    c = Category.objects.get(name__iregex=rpat)
    print(c)
except:
    print("no")

[0-9]{2}.0 Terrestrial and freshwater($|<)
no


In [91]:
for l in [4,6]:
    for i, c in enumerate(Category.objects.filter(name__iregex=rpat,level=l).order_by('id')):
        print(c.name, c.docusercat_set.count())
        if c.docusercat_set.count()==0 and i>0:
            pass
            #print(c.delete())

62 Armed conflict<hidden> 0
62 Armed conflict<hidden> 0
(5, {'scoping.Category_equivalents': 4, 'scoping.DocPar_category': 0, 'scoping.DocStatement_category': 0, 'scoping.Category': 1})
62 Armed conflict<hidden> 0
62 Armed conflict<hidden> 0
(5, {'scoping.Category_equivalents': 4, 'scoping.DocPar_category': 0, 'scoping.DocStatement_category': 0, 'scoping.Category': 1})


In [86]:
c2 = Category.objects.filter(name__iregex=rpat,level=4).order_by('id')[1]
c1 = Category.objects.filter(name__iregex=rpat,level=4).order_by('id')[0]

print(c1.docusercat_set.count())
print(c2.docusercat_set.count())

dc = c2.docusercat_set.last()
dc.category=c1
dc.save()

63
1


In [59]:
print(c1.level,c2.level)

4 6


In [7]:
titles = Category.objects.filter(title_only=True)
titles.values('name')

<QuerySet [{'name': '29.0 Marine & coastal'}, {'name': '42.0 Terrestrial and freshwater'}, {'name': '42.0 Terrestrial and freshwater'}, {'name': '29.0 Marine & coastal'}]>

In [10]:
cs = Category.objects.filter(title_only=True,name__icontains="Marine")
for c in cs:
    c.name = "29.0 Marine & coastal"
    c.save()

In [8]:
cs = Category.objects.filter(title_only=True,name__icontains="Terrestrial")
for c in cs:
    c.name = "42.0 Terrestrial and freshwater"
    c.save()